In [9]:
# Step 2: Write code here
import pandas as pd
import folium
from IPython.display import display, HTML
from ipywidgets import interact_manual
import requests
import json
import matplotlib.pyplot as plt
!pip install nba_api
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats


# Get Player_ID Through NBA API data system
def get_player_id(player_name):
    nba_df = pd.DataFrame()
    nba_players = players.get_players()
    nba_df = nba_df._append(nba_players)
    
    player_names_id = nba_df[['full_name', 'id']]
    
    try:
        player_id = player_names_id[player_names_id['full_name'] == player_name]['id'].values[0]
        return player_id
    except IndexError:
        print(f"Player '{player_name}' not found in the DataFrame.")
        return None
    
#Build Team List    
def get_team_list():
    team_df = pd.read_csv('https://raw.githubusercontent.com/danielfrg/espn-nba-scrapy/master/data/teams.csv')
    team_df[['prefix_1', 'name']]
    team_columns = team_df[['prefix_1', 'name']].copy()
    team_columns['prefix_1'] = team_columns['prefix_1'].str.upper()
    unique_sorted_teams = team_columns.drop_duplicates().sort_values(by=['prefix_1'])
    team_list = unique_sorted_teams.values.tolist()
    abbreviation_mapping = {'GS':'GSW','NO': 'NOP', 'NY': 'NYK', 'SA':'SAS'}
    for team in team_list:
        if team[0] in abbreviation_mapping:
            team[0] = abbreviation_mapping[team[0]]
    return team_list

#Get Teams Player Played On
def get_team_names(player_id):
    career = playercareerstats.PlayerCareerStats(player_id)
    teams_abbreviations = career.get_data_frames()[0]['TEAM_ABBREVIATION'].unique().tolist()
    team_dict = get_team_list()
    team_dict_lookup = {abbr: team_name for abbr, team_name in team_dict}
    matched_team_names = [team_dict_lookup[abbr] for abbr in teams_abbreviations if abbr in team_dict_lookup]
    matched_team_names.sort()
    return matched_team_names


#Load Locations
def load_geojson():
    url = 'https://raw.githubusercontent.com/sisskind/GeoJSON-NBA/master/arenas.geojson'
    response = requests.get(url)
    response.raise_for_status()
    geojson_data = response.json()
    return geojson_data

#Get Location of these teams Stadiums
def get_coordinates(matched_team_names):
    coordinates_list = []
    arena_df = pd.json_normalize(geojson_data['features'])
    arena_df['Team Name'] = arena_df['properties.city'] + " " + arena_df['properties.team']
    stadium_df = arena_df[['geometry.coordinates', 'Team Name', 'properties.arena']]
    stadium_df = stadium_df.sort_values(by='Team Name').reset_index(drop=True)
    stadium_df['Team Name'] = stadium_df['Team Name'].replace('Charlotte Hornets', 'Charlotte Bobcats')
    for index, row in stadium_df.iterrows():
        if row['Team Name'] in matched_team_names:
            lat = row['geometry.coordinates'][1]
            lon = row['geometry.coordinates'][0]
            coordinates_list.append((lat, lon))
    return coordinates_list

#Create Map

def create_map(team_locations):
    
    map_with_markers = folium.Map(location=[37.09024, -95.712891], zoom_start=4)

    for team_name, (lat, lon) in team_locations.items():
        popup_content = f"{team_name}"
        folium.Marker(location=[lat, lon], popup=folium.Popup(popup_content, parse_html=True)).add_to(map_with_markers)

    return map_with_markers


#Bing Search API Gets the link for the Image of the NBA player with the respective team they are on 
def get_player_image(api_key, player_name, matched_team_names):
    endpoint = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {"Ocp-Apim-Subscription-Key": api_key}
    
    for team_name in matched_team_names:
        query = f'{player_name} when he was on {team_name}'
        params = {
            "q": query,
            "mkt": "en-US",
            "safeSearch": "Moderate",
        }

        response = requests.get(endpoint, headers=headers, params=params)
        search_results = response.json()

        if "value" in search_results and len(search_results["value"]) > 0:
            image_url = search_results["value"][0]["contentUrl"]
            print(f"Image for {player_name} when he was on {team_name}: {image_url}")
        else:
            print(f"No images found for {player_name} when he was on {team_name}.")

#Bar Graph that shows Player Points Per Season
def create_bar(df):
    fig, ax = plt.subplots()
    df.plot(kind='bar', x='Season_Age', y='PTS', legend=False, ax=ax)
    
    ax.set_xlabel('Season/Age')
    ax.set_ylabel('Points')
    ax.set_title('Player Points In Season')

    fig.tight_layout()

    return fig
#Bar Graph that shows Player Minutes Per Season
def create_bar_minutes(df):
    fig2, ax = plt.subplots()
    df.plot(kind='bar', x='Season_Age', y='MIN', legend=False, ax=ax)
    
    ax.set_xlabel('Season/Age')
    ax.set_ylabel('Minutes')
    ax.set_title('Player Minutes In Season')

    fig2.tight_layout()

    return fig2
#Line Graph that shows the Correlation between Payer's points asnd the team they played on
def create_line(df):
    fig3, ax = plt.subplots()
    for team in df['TEAM_ABBREVIATION'].unique():
        team_data = df[df['TEAM_ABBREVIATION'] == team]
        ax.plot(team_data['Season_Age'], team_data['PTS'], marker='o', label=team)


    ax.set_xlabel('Season/Age')
    ax.set_ylabel('Points')
    ax.set_title('Player Points Over Time with Team Changes')

    ax.legend()
    
    plt.xticks(rotation=45, ha='right')

    return fig3

team_list = get_team_list()

geojson_data = load_geojson()

api_key = 'a047ee70a2394280939dfaf0691c787c'

#Main Function
display(HTML("<h1>NBA Player Career Analytics Tool</h1>"))
@interact_manual(player_name ="")
def player_info(player_name):
    player_id = get_player_id(player_name)
    
    matched_team_names = get_team_names(player_id)
    
    coordinates_list = get_coordinates(matched_team_names)
    team_locations = {team_name: coordinates for team_name, coordinates in zip(matched_team_names, coordinates_list)}
    map_with_markers = create_map(team_locations)
    map_with_markers.save(f'{player_name} map.html')
    
    player_image = get_player_image(api_key, player_name, matched_team_names)
    
    career = playercareerstats.PlayerCareerStats(player_id)
    career_df = career.get_data_frames()[0]
    career_df['Season_Age'] = 'Season: ' + career_df['SEASON_ID'] + ' ' + 'AGE: ' + career_df['PLAYER_AGE'].astype(str)
    
    points_bar = create_bar(career_df).savefig(f'{player_name}_points.jpeg')
    minutes_bar = create_bar_minutes(career_df).savefig(f'{player_name}_minutes.jpeg')
    team_versus_points_line = create_line(career_df).savefig(f'{player_name}_line.jpeg')

interactive(children=(Text(value='', continuous_update=False, description='player_name'), Button(description='…